# Convolutional Sentiment Classifier

In this notebook, we build a *convolutional* neural net to classify IMDB movie reviews by their sentiment.

#### Load dependencies

In [1]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Embedding
from keras.layers import Conv1D, MaxPooling1D # new! 

Using TensorFlow backend.


#### Set hyperparameters

In [2]:
# training:
epochs = 10
batch_size = 128

# vector-space embedding: 
n_dim = 64
n_unique_words = 5000 # as per Maas et al. (2011); may not be optimal
n_words_to_skip = 50 # ditto
max_review_length = 100
pad_type = trunc_type = 'pre'

# convolutional layer architecture:
n_conv = 128 # filters
k_conv = 3 # kernel length

# dense layer architecture: 
n_dense = 64
dropout = 0.5

#### Load data

In [3]:
(x_train, y_train), (x_valid, y_valid) = imdb.load_data(num_words=n_unique_words, skip_top=n_words_to_skip) 

#### Preprocess data

In [4]:
x_train = pad_sequences(x_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
x_valid = pad_sequences(x_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

#### Design neural network architecture

In [5]:
model = Sequential()
model.add(Embedding(n_unique_words, n_dim, input_length=max_review_length)) # remove input_length
model.add(Conv1D(n_conv, k_conv, activation='relu'))
# model.add(MaxPooling1D(pool_size=2)) # doesn't help or hurt
model.add(Flatten())
model.add(Dense(n_dense, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))

In [6]:
model.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 64)           320000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 128)           24704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 49, 128)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                401472    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total para

#### Configure model

In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Train!

In [8]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_valid, y_valid))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 8s - loss: 0.5110 - acc: 0.7248 - val_loss: 0.3442 - val_acc: 0.8463
Epoch 2/10
25000/25000 [==============================] - 8s - loss: 0.3132 - acc: 0.8741 - val_loss: 0.3471 - val_acc: 0.8463
Epoch 3/10
25000/25000 [==============================] - 8s - loss: 0.2595 - acc: 0.8998 - val_loss: 0.3637 - val_acc: 0.8449
Epoch 4/10
25000/25000 [==============================] - 8s - loss: 0.2055 - acc: 0.9243 - val_loss: 0.3976 - val_acc: 0.8417
Epoch 5/10
25000/25000 [==============================] - 8s - loss: 0.1409 - acc: 0.9512 - val_loss: 0.4550 - val_acc: 0.8340
Epoch 6/10
25000/25000 [==============================] - 8s - loss: 0.0734 - acc: 0.9792 - val_loss: 0.6008 - val_acc: 0.8297
Epoch 7/10
25000/25000 [==============================] - 8s - loss: 0.0314 - acc: 0.9924 - val_loss: 0.7406 - val_acc: 0.8291
Epoch 8/10
25000/25000 [==============================] - 8s 